# Test Results Review Report

This helps create the milestone report.

## Setup

First we need to load the JDP library which does the heavy lifting; importing and transforming the test result data from OpenQA into something useable. This assumes you started this notebook by running `julia src/notebook.jl`

In [16]:
import Pkg

using DataFrames

# Assumes you started the notebook with jdp/src/notebook.jl
Pkg.activate("../")

using JDP
using JDP.OpenQA
using JDP.TableDB

Next we set some variables so that we get the data from the correct location

In [14]:
datadir = "/home/rich/qa/data/osd" # The cache dir for the OpenQA test result data
cache_type = :binary # Set to :json to use the raw JSON data from OpenQA

:binary

Now we load the data into memory, this can take a while

In [17]:
json = nothing
df = nothing

if cache_type == :binary
    df = TableDB.load_module_results(joinpath(datadir, "cache.jld2"))
else
    json = OpenQA.load_job_results_json(datadir)
    df = TableDB.get_module_results(json)
end

"Loaded $(nrow(df)) results"

┌ Info: Recompiling stale cache file /home/rich/.julia/compiled/v1.0/JLD2/O1EyT.ji for JLD2 [033835bb-8acc-5ee8-8aae-3f567f8a3819]
└ @ Base loading.jl:1184


,variable,mean,min,median,max,nunique,nmissing,eltype
1,build,,0250,,0393,41,,String
2,name,,1_fs_stress,,xfs-490,2400,,String
3,result,,canceled,,softfailed,5,,String
4,arch,,aarch64,,x86_64,4,,String
5,suit,,"(""LTP"", ""cve"")",,"(""fstests"", ""xfs"")",22,,"Tuple{String,Union{Missing, String}}"
6,bugrefs,,[],,"[""t#2009216""]",151,,"Array{SubString,N} where N"


The function `describe` from the DataFrames package gives us some stats and information about the structure of the loaded data

In [23]:
describe(df, stats = [:nunique, :min, :max, :eltype])

,variable,nunique,min,max,eltype
1,build,41,0250,0393,String
2,name,2400,1_fs_stress,xfs-490,String
3,result,5,canceled,softfailed,String
4,arch,4,aarch64,x86_64,String
5,suit,22,"(""LTP"", ""cve"")","(""fstests"", ""xfs"")","Tuple{String,Union{Missing, String}}"
6,bugrefs,151,[],"[""t#2009216""]","Array{SubString,N} where N"


## Failed tests for build

In [28]:
build = "0393"

filter(r -> r[:build] == build && r[:result] == "failed", df)

,build,name,result,arch,suit,bugrefs
1,0393,run,failed,aarch64,"(""fstests"", ""btrfs"")","[""poo#40205"", ""t#1977181""]"
2,0393,generate_report,failed,aarch64,"(""fstests"", ""btrfs"")","[""poo#40205"", ""t#1977181""]"
3,0393,btrfs-078,failed,aarch64,"(""fstests"", ""btrfs"")",[]
4,0393,btrfs-152,failed,aarch64,"(""fstests"", ""btrfs"")",[]
5,0393,btrfs-159,failed,aarch64,"(""fstests"", ""btrfs"")",[]
6,0393,btrfs-168,failed,aarch64,"(""fstests"", ""btrfs"")",[]
7,0393,btrfs-170,failed,aarch64,"(""fstests"", ""btrfs"")",[]
8,0393,generic-122,failed,aarch64,"(""fstests"", ""btrfs"")",[]
9,0393,generic-136,failed,aarch64,"(""fstests"", ""btrfs"")",[]
10,0393,generic-175,failed,aarch64,"(""fstests"", ""btrfs"")",[]
